In [2]:
import IPython, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa
%matplotlib inline

import audio_utils

x = audio_utils.make_sinewave(440, 3, 22050)
audio_utils.play_audio(x, 22050)